<a href="https://colab.research.google.com/github/Harris-giki/Classification_Transformer_Models/blob/main/MVSwinTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.5 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.utils.checkpoint as checkpoint
from timm.models.layers import trunc_normal_, to_2tuple
import torch.nn.functional as F
from timm.models.layers import DropPath
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt

class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.ReLU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

def window_partition(x, window_size):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): window size

    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size,
               W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous(
    ).view(-1, window_size, window_size, C)
    return windows


def window_reverse(windows, window_size, H, W):
    """
    Args:
        windows: (num_windows*B, window_size, window_size, C)
        window_size (int): Window size
        H (int): Height of image
        W (int): Width of image

    Returns:
        x: (B, H, W, C)
    """
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size,
                     window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    return x

class WindowAttention_singleview(nn.Module):
    r""" Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.

    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
        pretrained_window_size (tuple[int]): The height and width of the window in pre-training.
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=True, attn_drop=0., proj_drop=0.,
                 pretrained_window_size=[0, 0]):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww
        self.pretrained_window_size = pretrained_window_size
        self.num_heads = num_heads

        self.logit_scale = nn.Parameter(
            torch.log(10 * torch.ones((num_heads, 1, 1))), requires_grad=True)

        self.device = torch.device(
            "cuda:0" if torch.cuda.is_available() else "cpu")

        # mlp to generate continuous relative position bias
        self.cpb_mlp = nn.Sequential(nn.Linear(2, 512, bias=True),
                                     nn.ReLU(inplace=True),
                                     nn.Linear(512, num_heads, bias=False))

        # get relative_coords_table
        relative_coords_h = torch.arange(
            -(self.window_size[0] - 1), self.window_size[0], dtype=torch.float32)
        relative_coords_w = torch.arange(
            -(self.window_size[1] - 1), self.window_size[1], dtype=torch.float32)
        relative_coords_table = torch.stack(
            torch.meshgrid([relative_coords_h,
                            relative_coords_w])).permute(1, 2, 0).contiguous().unsqueeze(0)  # 1, 2*Wh-1, 2*Ww-1, 2
        if pretrained_window_size[0] > 0:
            relative_coords_table[:, :, :,
                                  0] /= (pretrained_window_size[0] - 1)
            relative_coords_table[:, :, :,
                                  1] /= (pretrained_window_size[1] - 1)
        else:
            relative_coords_table[:, :, :, 0] /= (self.window_size[0] - 1)
            relative_coords_table[:, :, :, 1] /= (self.window_size[1] - 1)
        relative_coords_table *= 8  # normalize to -8, 8
        relative_coords_table = (torch.sign(relative_coords_table) * torch.log2(
            torch.abs(relative_coords_table) + 1.0) / np.log2(8)).to(self.device)

        self.register_buffer("relative_coords_table", relative_coords_table)

        # get pair-wise relative position index for each token inside the window
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
        relative_coords = coords_flatten[:, :, None] - \
            coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
        relative_coords = relative_coords.permute(
            1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
        relative_coords[:, :, 0] += self.window_size[0] - \
            1  # shift to start from 0
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        self.register_buffer("relative_position_index",
                             relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=False)
        if qkv_bias:
            self.q_bias = nn.Parameter(torch.zeros(dim))
            self.v_bias = nn.Parameter(torch.zeros(dim))
        else:
            self.q_bias = None
            self.v_bias = None
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape
        qkv_bias = None
        if self.q_bias is not None:
            qkv_bias = torch.cat((self.q_bias, torch.zeros_like(
                self.v_bias, requires_grad=False), self.v_bias))
        qkv = F.linear(input=x, weight=self.qkv.weight, bias=qkv_bias)
        qkv = qkv.reshape(B_, N, 3, self.num_heads, -1).permute(2, 0, 3, 1, 4)
        # make torchscript happy (cannot use tensor as tuple)
        q, k, v = qkv[0], qkv[1], qkv[2]

        # cosine attention
        attn = (F.normalize(q, dim=-1) @
                F.normalize(k, dim=-1).transpose(-2, -1))
        logit_scale = torch.clamp(self.logit_scale, max=torch.log(
            torch.tensor(1. / 0.01).cuda())).exp()
        attn = attn * logit_scale

        relative_position_bias_table = self.cpb_mlp(
            self.relative_coords_table).view(-1, self.num_heads)
        relative_position_bias = relative_position_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)  # Wh*Ww,Wh*Ww,nH
        relative_position_bias = relative_position_bias.permute(
            2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
        relative_position_bias = 16 * torch.sigmoid(relative_position_bias)
        attn = attn + relative_position_bias.unsqueeze(0)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N,
                             N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

    def extra_repr(self) -> str:
        return f'dim={self.dim}, window_size={self.window_size}, ' \
               f'pretrained_window_size={self.pretrained_window_size}, num_heads={self.num_heads}'

    def flops(self, N):
        # calculate flops for 1 window with token length of N
        flops = 0
        # qkv = self.qkv(x)
        flops += N * self.dim * 3 * self.dim
        # attn = (q @ k.transpose(-2, -1))
        flops += self.num_heads * N * (self.dim // self.num_heads) * N
        #  x = (attn @ v)
        flops += self.num_heads * N * N * (self.dim // self.num_heads)
        # x = self.proj(x)
        flops += N * self.dim * self.dim
        return flops


class SwinTransformerBlock_singleview(nn.Module):
    r""" Swin Transformer Block.

    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resulotion.
        num_heads (int): Number of attention heads.
        window_size (int): Window size.
        shift_size (int): Shift size for SW-MSA.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float, optional): Stochastic depth rate. Default: 0.0
        act_layer (nn.Module, optional): Activation layer. Default: nn.GELU
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
        pretrained_window_size (int): Window size in pre-training.
    """

    def __init__(self, dim, input_resolution, num_heads, window_size=12, shift_size=0,
                 mlp_ratio=4., qkv_bias=True, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.ReLU, norm_layer=nn.LayerNorm, pretrained_window_size=0):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        if min(self.input_resolution) <= self.window_size:
            # if window size is larger than input resolution, we don't partition windows
            self.shift_size = 0
            self.window_size = min(self.input_resolution)
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"

        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention_singleview(
            dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop,
            pretrained_window_size=to_2tuple(pretrained_window_size))

        self.drop_path = DropPath(
            drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim,
                       act_layer=act_layer, drop=drop)

        if self.shift_size > 0:
            # calculate attention mask for SW-MSA
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1

            # nW, window_size, window_size, 1
            mask_windows = window_partition(img_mask, self.window_size)
            mask_windows = mask_windows.view(-1,
                                             self.window_size * self.window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(
                attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None

        self.register_buffer("attn_mask", attn_mask)

    def forward(self, x):
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"

        shortcut = x
        x = x.view(B, H, W, C)

        # cyclic shift
        if self.shift_size > 0:
            shifted_x = torch.roll(
                x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
        else:
            shifted_x = x

        # partition windows
        # nW*B, window_size, window_size, C
        x_windows = window_partition(shifted_x, self.window_size)
        # nW*B, window_size*window_size, C
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)

        # W-MSA/SW-MSA
        # nW*B, window_size*window_size, C
        attn_windows = self.attn(x_windows, mask=self.attn_mask)

        # merge windows
        attn_windows = attn_windows.view(-1,
                                         self.window_size, self.window_size, C)
        shifted_x = window_reverse(
            attn_windows, self.window_size, H, W)  # B H' W' C

        # reverse cyclic shift
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(
                self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x
        x = x.view(B, H * W, C)
        x = shortcut + self.drop_path(self.norm1(x))

        # FFN
        x = x + self.drop_path(self.norm2(self.mlp(x)))

        return x

    def extra_repr(self) -> str:
        return f"dim={self.dim}, input_resolution={self.input_resolution}, num_heads={self.num_heads}, " \
               f"window_size={self.window_size}, shift_size={self.shift_size}, mlp_ratio={self.mlp_ratio}"

    def flops(self):
        flops = 0
        H, W = self.input_resolution
        # norm1
        flops += self.dim * H * W
        # W-MSA/SW-MSA
        nW = H * W / self.window_size / self.window_size
        flops += nW * self.attn.flops(self.window_size * self.window_size)
        # mlp
        flops += 2 * H * W * self.dim * self.dim * self.mlp_ratio
        # norm2
        flops += self.dim * H * W
        return flops


class BasicLayer_singleview(nn.Module):
    """ A basic single view Swin Transformer layer for one stage.

    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resolution.
        depth (int): Number of blocks.
        num_heads (int): Number of attention heads.
        window_size (int): Local window size.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float | tuple[float], optional): Stochastic depth rate. Default: 0.0
        norm_layer (nn.Module, optional): Normalization layer. Default: nn.LayerNorm
        downsample (nn.Module | None, optional): Downsample layer at the end of the layer. Default: None
        use_checkpoint (bool): Whether to use checkpointing to save memory. Default: False.
        pretrained_window_size (int): Local window size in pre-training.
    """

    def __init__(self, dim, input_resolution, depth, num_heads, window_size,
                 mlp_ratio=4., qkv_bias=True, drop=0., attn_drop=0.,
                 drop_path=0., norm_layer=nn.LayerNorm, downsample=None, use_checkpoint=False,
                 pretrained_window_size=0):

        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.depth = depth
        self.use_checkpoint = use_checkpoint

        # build blocks
        self.blocks = nn.ModuleList([
            SwinTransformerBlock_singleview(dim=dim, input_resolution=input_resolution,
                                          num_heads=num_heads, window_size=window_size,
                                          shift_size=0 if (
                                              i % 2 == 0) else window_size // 2,
                                          mlp_ratio=mlp_ratio,
                                          qkv_bias=qkv_bias,
                                          drop=drop, attn_drop=attn_drop,
                                          drop_path=drop_path[i] if isinstance(
                                              drop_path, list) else drop_path,
                                          norm_layer=norm_layer,
                                          pretrained_window_size=pretrained_window_size)
            for i in range(depth)])

        # patch merging layer
        if downsample is not None:
            self.downsample = downsample(
                input_resolution, dim=dim, norm_layer=norm_layer)
        else:
            self.downsample = None

    def forward(self, x):
        for blk in self.blocks:
            if self.use_checkpoint:
                x = checkpoint.checkpoint(blk, x)
            else:
                x = blk(x)
        if self.downsample is not None:
            x = self.downsample(x)
        return x

    def extra_repr(self) -> str:
        return f"dim={self.dim}, input_resolution={self.input_resolution}, depth={self.depth}"

    def flops(self):
        flops = 0
        for blk in self.blocks:
            flops += blk.flops()
        if self.downsample is not None:
            flops += self.downsample.flops()
        return flops

    def _init_respostnorm(self):
        for blk in self.blocks:
            nn.init.constant_(blk.norm1.bias, 0)
            nn.init.constant_(blk.norm1.weight, 0)
            nn.init.constant_(blk.norm2.bias, 0)
            nn.init.constant_(blk.norm2.weight, 0)

class DynamicAttention_multiview(nn.Module):
    r""" Window based multi-head self and cross attention (W-MDA) module
    for multi view mammograms with relative position bias.
    It supports both of shifted and non-shifted window.

    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
        pretrained_window_size (tuple[int]): The height and width of the window in pre-training.
        diff_attn (bool, optional): If True, use dynamic attention for different views. Default: True
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=True, attn_drop=0., proj_drop=0.,
                 pretrained_window_size=[0, 0], diff_attn=True):

        super().__init__()
        self.dim = dim
        self.window_size = window_size
        self.pretrained_window_size = pretrained_window_size
        self.num_heads = num_heads
        self.diff_attn = diff_attn

        # For first view
        self.logit_scale_1 = nn.Parameter(
            torch.log(10 * torch.ones((num_heads, 1, 1))), requires_grad=True)

        # mlp to generate continuous relative position bias
        self.cpb_mlp_1 = nn.Sequential(nn.Linear(2, 512, bias=True),
                                       nn.ReLU(inplace=True),
                                       nn.Linear(512, num_heads, bias=False))

        # get relative_coords_table
        relative_coords_h_1 = torch.arange(
            -(self.window_size[0] - 1), self.window_size[0], dtype=torch.float32)
        relative_coords_w_1 = torch.arange(
            -(self.window_size[1] - 1), self.window_size[1], dtype=torch.float32)
        relative_coords_table_1 = torch.stack(
            torch.meshgrid([relative_coords_h_1,
                            relative_coords_w_1])).permute(1, 2, 0).contiguous().unsqueeze(0)  # 1, 2*Wh-1, 2*Ww-1, 2
        if pretrained_window_size[0] > 0:
            relative_coords_table_1[:, :, :,
                                    0] /= (pretrained_window_size[0] - 1)
            relative_coords_table_1[:, :, :,
                                    1] /= (pretrained_window_size[1] - 1)
        else:
            relative_coords_table_1[:, :, :, 0] /= (self.window_size[0] - 1)
            relative_coords_table_1[:, :, :, 1] /= (self.window_size[1] - 1)
        relative_coords_table_1 *= 8  # normalize to -8, 8
        self.relative_coords_table_1 = (torch.sign(relative_coords_table_1) * torch.log2(
            torch.abs(relative_coords_table_1) + 1.0) / np.log2(8)).cuda()

        self.register_buffer("relative_coords_table",
                             self.relative_coords_table_1)

        # get pair-wise relative position index for each token inside the window
        coords_h_1 = torch.arange(self.window_size[0])
        coords_w_1 = torch.arange(self.window_size[1])
        coords_1 = torch.stack(torch.meshgrid(
            [coords_h_1, coords_w_1]))
        coords_flatten_1 = torch.flatten(coords_1, 1)

        relative_coords_1 = coords_flatten_1[:,
                                             :, None] - coords_flatten_1[:, None, :]
        relative_coords_1 = relative_coords_1.permute(
            1, 2, 0).contiguous()
        relative_coords_1[:, :, 0] += self.window_size[0] - \
            1
        relative_coords_1[:, :, 1] += self.window_size[1] - 1
        relative_coords_1[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index_1 = relative_coords_1.sum(-1)
        self.register_buffer("relative_position_index_1",
                             relative_position_index_1)

        self.qkv_1 = nn.Linear(dim, dim * 3, bias=False)
        if qkv_bias:
            self.q_bias_1 = nn.Parameter(torch.zeros(dim))
            self.v_bias_1 = nn.Parameter(torch.zeros(dim))
        else:
            self.q_bias_1 = None
            self.v_bias_1 = None
        self.attn_drop_1 = nn.Dropout(attn_drop)
        self.proj_1 = nn.Linear(dim, dim)
        self.proj_drop_1 = nn.Dropout(proj_drop)
        self.softmax_1 = nn.Softmax(dim=-1)

        # For second view

        self.logit_scale_2 = nn.Parameter(
            torch.log(10 * torch.ones((num_heads, 1, 1))), requires_grad=True)

        # mlp to generate continuous relative position bias
        self.cpb_mlp_2 = nn.Sequential(nn.Linear(2, 512, bias=True),
                                       nn.ReLU(inplace=True),
                                       nn.Linear(512, num_heads, bias=False))

        # get relative_coords_table
        relative_coords_h_2 = torch.arange(
            -(self.window_size[0] - 1), self.window_size[0], dtype=torch.float32)
        relative_coords_w_2 = torch.arange(
            -(self.window_size[1] - 1), self.window_size[1], dtype=torch.float32)
        relative_coords_table_2 = torch.stack(
            torch.meshgrid([relative_coords_h_2,
                            relative_coords_w_2])).permute(1, 2, 0).contiguous().unsqueeze(0)
        if pretrained_window_size[0] > 0:
            relative_coords_table_2[:, :, :,
                                    0] /= (pretrained_window_size[0] - 1)
            relative_coords_table_2[:, :, :,
                                    1] /= (pretrained_window_size[1] - 1)
        else:
            relative_coords_table_2[:, :, :, 0] /= (self.window_size[0] - 1)
            relative_coords_table_2[:, :, :, 1] /= (self.window_size[1] - 1)
        relative_coords_table_2 *= 8  # normalize to -8, 8
        self.relative_coords_table_2 = (torch.sign(relative_coords_table_2) * torch.log2(
            torch.abs(relative_coords_table_2) + 1.0) / np.log2(8)).cuda()

        self.register_buffer("relative_coords_table",
                             self.relative_coords_table_2)

        # get pair-wise relative position index for each token inside the window
        coords_h_2 = torch.arange(self.window_size[0])
        coords_w_2 = torch.arange(self.window_size[1])
        coords_2 = torch.stack(torch.meshgrid(
            [coords_h_2, coords_w_2]))
        coords_flatten_2 = torch.flatten(coords_2, 1)
        relative_coords_2 = coords_flatten_2[:,
                                             :, None] - coords_flatten_2[:, None, :]
        relative_coords_2 = relative_coords_2.permute(
            1, 2, 0).contiguous()
        relative_coords_2[:, :, 0] += self.window_size[0] - \
            1
        relative_coords_2[:, :, 1] += self.window_size[1] - 1
        relative_coords_2[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index_2 = relative_coords_2.sum(-1)  #
        self.register_buffer("relative_position_index_2",
                             relative_position_index_2)

        self.qkv_2 = nn.Linear(dim, dim * 3, bias=False)
        if qkv_bias:
            self.q_bias_2 = nn.Parameter(torch.zeros(dim))
            self.v_bias_2 = nn.Parameter(torch.zeros(dim))
        else:
            self.q_bias_2 = None
            self.v_bias_2 = None
        self.attn_drop_2 = nn.Dropout(attn_drop)
        self.proj_2 = nn.Linear(dim, dim)
        self.proj_drop_2 = nn.Dropout(proj_drop)
        self.softmax_2 = nn.Softmax(dim=-1)

        # For concatanation
        if self.diff_attn:
            self.proj_concat_1 = nn.Linear(self.num_heads * 2, self.num_heads)
            self.proj_concat_2 = nn.Linear(self.num_heads * 2, self.num_heads)

    def forward(self, x1, x2, mask1=None, mask2=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x1.shape

        # First view
        qkv_bias_1 = None
        if self.q_bias_1 is not None:
            qkv_bias_1 = torch.cat((self.q_bias_1, torch.zeros_like(
                self.v_bias_1, requires_grad=False), self.v_bias_1))
        qkv_1 = F.linear(input=x1, weight=self.qkv_1.weight, bias=qkv_bias_1)
        qkv_1 = qkv_1.reshape(B_, N, 3, self.num_heads, -
                              1).permute(2, 0, 3, 1, 4)
        q_x1, k_x1, v_x1 = qkv_1[0], qkv_1[1], qkv_1[2]

        # Second view
        qkv_bias_2 = None
        if self.q_bias_2 is not None:
            qkv_bias_2 = torch.cat((self.q_bias_2, torch.zeros_like(
                self.v_bias_2, requires_grad=False), self.v_bias_2))
        qkv_2 = F.linear(input=x2, weight=self.qkv_2.weight, bias=qkv_bias_2)
        qkv_2 = qkv_2.reshape(B_, N, 3, self.num_heads, -
                              1).permute(2, 0, 3, 1, 4)
        q_x2, k_x2, v_x2 = qkv_2[0], qkv_2[1], qkv_2[2]

        if self.diff_attn:
            # cosine attention- first view
            attn_same_x1 = (F.normalize(q_x1, dim=-1) @
                            F.normalize(k_x1, dim=-1).transpose(-2, -1))
            attn_diff_x1 = (F.normalize(q_x1, dim=-1) @
                            F.normalize(k_x2, dim=-1).transpose(-2, -1))
            logit_scale_1 = torch.clamp(self.logit_scale_1, max=torch.log(
                torch.tensor(1. / 0.01).cuda())).exp()
            # print("before concat: ", attn_same_x1.shape, attn_diff_x1.shape)
            concat_attn_x1 = torch.concat(
                (attn_same_x1, attn_diff_x1), dim=1).permute(0, 2, 3, 1)
            # print("concat : ", concat_attn_x1.shape)

            ## For weighted average:
            # attn_x1 = (0.9 * attn_same_x1 + 0.1 * attn_diff_x1) * logit_scale_1

            ## For concatenation:
            attn_x1 = self.proj_concat_1(concat_attn_x1).permute(
                0, 3, 1, 2) * logit_scale_1

            # cosine attention- second view
            attn_same_x2 = (F.normalize(q_x2, dim=-1) @
                            F.normalize(k_x2, dim=-1).transpose(-2, -1))
            attn_diff_x2 = (F.normalize(q_x2, dim=-1) @
                            F.normalize(k_x1, dim=-1).transpose(-2, -1))
            logit_scale_2 = torch.clamp(self.logit_scale_2, max=torch.log(
                torch.tensor(1. / 0.01).cuda())).exp()
            ## For weighted average:
            # attn_x2 = (0.9 * attn_same_x2 + 0.1 * attn_diff_x2) * logit_scale_2

            ## For concatenation:
            concat_attn_x2 = torch.concat(
                (attn_same_x2, attn_diff_x2), dim=1).permute(0, 2, 3, 1)
            attn_x2 = self.proj_concat_2(concat_attn_x2).permute(
                0, 3, 1, 2) * logit_scale_2
        else:
            # cosine attention- first view
            attn_same_x1 = (F.normalize(q_x1, dim=-1) @
                            F.normalize(k_x1, dim=-1).transpose(-2, -1))
            logit_scale_1 = torch.clamp(self.logit_scale_1, max=torch.log(
                torch.tensor(1. / 0.01).cuda())).exp()
            attn_x1 = attn_same_x1 * logit_scale_1

            # cosine attention- second view
            attn_same_x2 = (F.normalize(q_x2, dim=-1) @
                            F.normalize(k_x2, dim=-1).transpose(-2, -1))
            logit_scale_2 = torch.clamp(self.logit_scale_2, max=torch.log(
                torch.tensor(1. / 0.01).cuda())).exp()
            attn_x2 = attn_same_x2 * logit_scale_2

        relative_position_bias_table_1 = self.cpb_mlp_1(
            self.relative_coords_table_1).view(-1, self.num_heads)
        relative_position_bias_1 = relative_position_bias_table_1[self.relative_position_index_1.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)
        relative_position_bias_1 = relative_position_bias_1.permute(
            2, 0, 1).contiguous()
        relative_position_bias_1 = 16 * torch.sigmoid(relative_position_bias_1)
        attn_x1 = attn_x1 + relative_position_bias_1.unsqueeze(0)

        relative_position_bias_table_2 = self.cpb_mlp_2(
            self.relative_coords_table_2).view(-1, self.num_heads)
        relative_position_bias_2 = relative_position_bias_table_2[self.relative_position_index_2.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)
        relative_position_bias_2 = relative_position_bias_2.permute(
            2, 0, 1).contiguous()
        relative_position_bias_2 = 16 * torch.sigmoid(relative_position_bias_2)
        attn_x2 = attn_x2 + relative_position_bias_2.unsqueeze(0)

        if mask1 is not None:
            nW = mask1.shape[0]
            attn_x1 = attn_x1.view(
                B_ // nW, nW, self.num_heads, N, N) + mask1.unsqueeze(1).unsqueeze(0)
            attn_x1 = attn_x1.view(-1, self.num_heads, N, N)
            attn_x1 = self.softmax_1(attn_x1)
        else:
            attn_x1 = self.softmax_1(attn_x1)

        if mask1 is not None:
            nW = mask1.shape[0]
            attn_x2 = attn_x2.view(
                B_ // nW, nW, self.num_heads, N, N) + mask2.unsqueeze(1).unsqueeze(0)
            attn_x2 = attn_x2.view(-1, self.num_heads, N, N)
            attn_x2 = self.softmax_2(attn_x2)
        else:
            attn_x2 = self.softmax_2(attn_x2)

        attn_x1 = self.attn_drop_1(attn_x1)
        x1 = (attn_x1 @ v_x1).transpose(1, 2).reshape(B_, N, C)
        x1 = self.proj_1(x1)
        x1 = self.proj_drop_1(x1)

        attn_x2 = self.attn_drop_2(attn_x2)
        x2 = (attn_x2 @ v_x2).transpose(1, 2).reshape(B_, N, C)
        x2 = self.proj_2(x2)
        x2 = self.proj_drop_2(x2)
        return x1, x2

    def extra_repr(self) -> str:
        return f'dim={self.dim}, window_size={self.window_size}, ' \
               f'pretrained_window_size={self.pretrained_window_size}, num_heads={self.num_heads}'


class OmniAttentionTransformerBlock_multiview(nn.Module):
    r""" Omni Attention Transformer Block.

    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resulotion.
        num_heads (int): Number of attention heads.
        window_size (int): Window size.
        shift_size (int): Shift size for SW-MSA.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float, optional): Stochastic depth rate. Default: 0.0
        act_layer (nn.Module, optional): Activation layer. Default: nn.GELU
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
        pretrained_window_size (int): Window size in pre-training.
        diff_attn (bool, optional): If True, use dynamic attention for different views. Default: True
    """

    def __init__(self, dim, input_resolution, num_heads, window_size=12, shift_size=0,
                 mlp_ratio=4., qkv_bias=True, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.ReLU, norm_layer=nn.LayerNorm, pretrained_window_size=0, diff_attn=True):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        if min(self.input_resolution) <= self.window_size:
            self.shift_size = 0
            self.window_size = min(self.input_resolution)
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"

        self.norm1_1 = norm_layer(dim)
        self.norm1_2 = norm_layer(dim)
        self.attn = DynamicAttention_multiview(
            dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop,
            pretrained_window_size=to_2tuple(pretrained_window_size), diff_attn=diff_attn)

        self.drop_path_1 = DropPath(
            drop_path) if drop_path > 0. else nn.Identity()
        self.drop_path_2 = DropPath(
            drop_path) if drop_path > 0. else nn.Identity()
        self.norm2_1 = norm_layer(dim)
        self.norm2_2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp_1 = Mlp(
            in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)
        self.mlp_2 = Mlp(
            in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        if self.shift_size > 0:
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1

            mask_windows = window_partition(img_mask, self.window_size)
            mask_windows = mask_windows.view(-1,
                                             self.window_size * self.window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(
                attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None

        self.register_buffer("attn_mask", attn_mask)

    def forward(self, x1, x2):
        H, W = self.input_resolution
        B, L, C = x1.shape
        assert L == H * W, "input feature has wrong size"

        shortcut_x1 = x1
        x1 = x1.view(B, H, W, C)
        shortcut_x2 = x2
        x2 = x2.view(B, H, W, C)

        # cyclic shift
        if self.shift_size > 0:
            shifted_x1 = torch.roll(
                x1, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
            shifted_x2 = torch.roll(
                x2, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
        else:
            shifted_x1 = x1
            shifted_x2 = x2

        x_windows_1 = window_partition(shifted_x1, self.window_size)
        x_windows_1 = x_windows_1.view(-1,
                                       self.window_size * self.window_size, C)

        x_windows_2 = window_partition(shifted_x2, self.window_size)
        x_windows_2 = x_windows_2.view(-1,
                                       self.window_size * self.window_size, C)

        # W-MDA/SW-MDA
        attn_windows_1, attn_windows_2 = self.attn(x_windows_1,
                                                   x_windows_2,
                                                   mask1=self.attn_mask,
                                                   mask2=self.attn_mask)

        # merge windows
        attn_windows_1 = attn_windows_1.view(-1,
                                             self.window_size, self.window_size, C)
        shifted_x1 = window_reverse(
            attn_windows_1, self.window_size, H, W)
        attn_windows_2 = attn_windows_2.view(-1,
                                             self.window_size, self.window_size, C)
        shifted_x2 = window_reverse(
            attn_windows_2, self.window_size, H, W)

        # reverse cyclic shift
        if self.shift_size > 0:
            x1 = torch.roll(shifted_x1, shifts=(
                self.shift_size, self.shift_size), dims=(1, 2))
            x2 = torch.roll(shifted_x2, shifts=(
                self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x1 = shifted_x1
            x2 = shifted_x2
        x1 = x1.view(B, H * W, C)
        x1 = shortcut_x1 + self.drop_path_1(self.norm1_1(x1))
        x2 = x2.view(B, H * W, C)
        x2 = shortcut_x2 + self.drop_path_2(self.norm1_2(x2))

        # FFN
        x1 = x1 + self.drop_path_1(self.norm2_1(self.mlp_1(x1)))
        x2 = x2 + self.drop_path_2(self.norm2_2(self.mlp_2(x2)))

        return x1, x2
class PatchMerging(nn.Module):
    r""" Patch Merging Layer.

    Args:
        input_resolution (tuple[int]): Resolution of input feature.
        dim (int): Number of input channels.
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, input_resolution, dim, norm_layer=nn.LayerNorm):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)
        self.norm = norm_layer(2 * dim)

    def forward(self, x):
        """
        x: B, H*W, C
        """
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"
        assert H % 2 == 0 and W % 2 == 0, f"x size ({H}*{W}) are not even."

        x = x.view(B, H, W, C)

        x0 = x[:, 0::2, 0::2, :]
        x1 = x[:, 1::2, 0::2, :]
        x2 = x[:, 0::2, 1::2, :]
        x3 = x[:, 1::2, 1::2, :]
        x = torch.cat([x0, x1, x2, x3], -1)
        x = x.view(B, -1, 4 * C)

        x = self.reduction(x)
        x = self.norm(x)

        return x

    def extra_repr(self) -> str:
        return f"input_resolution={self.input_resolution}, dim={self.dim}"

    def flops(self):
        H, W = self.input_resolution
        flops = (H // 2) * (W // 2) * 4 * self.dim * 2 * self.dim
        flops += H * W * self.dim // 2
        return flops

class PatchEmbed(nn.Module):
    r""" Image to Patch Embedding

    Args:
        img_size (int): Image size.  Default: 224.
        patch_size (int): Patch token size. Default: 4.
        in_chans (int): Number of input image channels. Default: 3.
        embed_dim (int): Number of linear projection output channels. Default: 96.
        norm_layer (nn.Module, optional): Normalization layer. Default: None
    """

    def __init__(self, img_size=384, patch_size=4, in_chans=3, embed_dim=96, norm_layer=None):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        patches_resolution = [img_size[0] //
                              patch_size[0], img_size[1] // patch_size[1]]
        self.img_size = img_size
        self.patch_size = patch_size
        self.patches_resolution = patches_resolution
        self.num_patches = patches_resolution[0] * patches_resolution[1]

        self.in_chans = in_chans
        self.embed_dim = embed_dim

        self.proj = nn.Conv2d(in_chans, embed_dim,
                              kernel_size=patch_size, stride=patch_size)
        if norm_layer is not None:
            self.norm = norm_layer(embed_dim)
        else:
            self.norm = None

    def forward(self, x):
        B, C, H, W = x.shape
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x = self.proj(x).flatten(2).transpose(1, 2)
        if self.norm is not None:
            x = self.norm(x)
        return x

    def flops(self):
        Ho, Wo = self.patches_resolution
        flops = Ho * Wo * self.embed_dim * self.in_chans * \
            (self.patch_size[0] * self.patch_size[1])
        if self.norm is not None:
            flops += Ho * Wo * self.embed_dim
        return flops

class BasicLayer_multiview(nn.Module):
    """ A basic Multi view Transformer layer for one stage.

    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resolution.
        depth (int): Number of blocks.
        num_heads (int): Number of attention heads.
        window_size (int): Local window size.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float | tuple[float], optional): Stochastic depth rate. Default: 0.0
        norm_layer (nn.Module, optional): Normalization layer. Default: nn.LayerNorm
        downsample (nn.Module | None, optional): Downsample layer at the end of the layer. Default: None
        use_checkpoint (bool): Whether to use checkpointing to save memory. Default: False.
        pretrained_window_size (int): Local window size in pre-training.
    """

    def __init__(self, dim, input_resolution, depth, num_heads, window_size,
                 mlp_ratio=4., qkv_bias=True, drop=0., attn_drop=0.,
                 drop_path=0., norm_layer=nn.LayerNorm, downsample=None, use_checkpoint=False,
                 pretrained_window_size=0, diff_attn=True):

        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.depth = depth
        self.use_checkpoint = use_checkpoint

        # build blocks
        self.blocks = nn.ModuleList([
            OmniAttentionTransformerBlock_multiview(dim=dim, input_resolution=input_resolution,
                                 num_heads=num_heads, window_size=window_size,
                                 shift_size=0 if (
                                     i % 2 == 0) else window_size // 2,
                                 mlp_ratio=mlp_ratio,
                                 qkv_bias=qkv_bias,
                                 drop=drop, attn_drop=attn_drop,
                                 drop_path=drop_path[i] if isinstance(
                                     drop_path, list) else drop_path,
                                 norm_layer=norm_layer,
                                 pretrained_window_size=pretrained_window_size,
                                 diff_attn=diff_attn)
            for i in range(depth)])

        # patch merging layer
        if downsample is not None:
            self.downsample_1 = downsample(
                input_resolution, dim=dim, norm_layer=norm_layer)
            self.downsample_2 = downsample(
                input_resolution, dim=dim, norm_layer=norm_layer)
        else:
            self.downsample_1 = None

    def forward(self, x1, x2):
        for blk in self.blocks:
            if self.use_checkpoint:
                x1, x2 = checkpoint.checkpoint(blk, x1, x2)
            else:
                x1, x2 = blk(x1, x2)
        if self.downsample_1 is not None:
            x1 = self.downsample_1(x1)
            x2 = self.downsample_2(x2)
        return x1, x2

    def _init_respostnorm(self):
        for blk in self.blocks:
            nn.init.constant_(blk.norm1_1.bias, 0)
            nn.init.constant_(blk.norm1_1.weight, 0)
            nn.init.constant_(blk.norm2_1.bias, 0)
            nn.init.constant_(blk.norm2_1.weight, 0)
            nn.init.constant_(blk.norm1_2.bias, 0)
            nn.init.constant_(blk.norm1_2.weight, 0)
            nn.init.constant_(blk.norm2_2.bias, 0)
            nn.init.constant_(blk.norm2_2.weight, 0)


class BasicLayer_singleview(nn.Module):
    """ A basic single view Swin Transformer layer for one stage.

    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resolution.
        depth (int): Number of blocks.
        num_heads (int): Number of attention heads.
        window_size (int): Local window size.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float | tuple[float], optional): Stochastic depth rate. Default: 0.0
        norm_layer (nn.Module, optional): Normalization layer. Default: nn.LayerNorm
        downsample (nn.Module | None, optional): Downsample layer at the end of the layer. Default: None
        use_checkpoint (bool): Whether to use checkpointing to save memory. Default: False.
        pretrained_window_size (int): Local window size in pre-training.
    """

    def __init__(self, dim, input_resolution, depth, num_heads, window_size,
                 mlp_ratio=4., qkv_bias=True, drop=0., attn_drop=0.,
                 drop_path=0., norm_layer=nn.LayerNorm, downsample=None, use_checkpoint=False,
                 pretrained_window_size=0):

        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.depth = depth
        self.use_checkpoint = use_checkpoint

        # build blocks
        self.blocks = nn.ModuleList([
            SwinTransformerBlock_singleview(dim=dim, input_resolution=input_resolution,
                                          num_heads=num_heads, window_size=window_size,
                                          shift_size=0 if (
                                              i % 2 == 0) else window_size // 2,
                                          mlp_ratio=mlp_ratio,
                                          qkv_bias=qkv_bias,
                                          drop=drop, attn_drop=attn_drop,
                                          drop_path=drop_path[i] if isinstance(
                                              drop_path, list) else drop_path,
                                          norm_layer=norm_layer,
                                          pretrained_window_size=pretrained_window_size)
            for i in range(depth)])

        # patch merging layer
        if downsample is not None:
            self.downsample = downsample(
                input_resolution, dim=dim, norm_layer=norm_layer)
        else:
            self.downsample = None

    def forward(self, x):
        for blk in self.blocks:
            if self.use_checkpoint:
                x = checkpoint.checkpoint(blk, x)
            else:
                x = blk(x)
        if self.downsample is not None:
            x = self.downsample(x)
        return x

    def _init_respostnorm(self):
        for blk in self.blocks:
            nn.init.constant_(blk.norm1.bias, 0)
            nn.init.constant_(blk.norm1.weight, 0)
            nn.init.constant_(blk.norm2.bias, 0)
            nn.init.constant_(blk.norm2.weight, 0)



class MVSwinTransformer(nn.Module):
    r""" Multiview Swin Transformer
        A PyTorch impl of : `MV-Swin-T: Mammogram Classification with Multi-View Swin Transformer`
    Args:
        img_size (int | tuple(int)): Input image size. Default 224
        patch_size (int | tuple(int)): Patch size. Default: 4
        in_chans (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1
        embed_dim (int): Patch embedding dimension. Default: 96
        depths (tuple(int)): Depth of each Swin Transformer layer.
        num_heads (tuple(int)): Number of attention heads in different layers.
        window_size (int): Window size. Default: 12 (for 384x384 input image)
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim. Default: 4
        qkv_bias (bool): If True, add a learnable bias to query, key, value. Default: True
        drop_rate (float): Dropout rate. Default: 0
        attn_drop_rate (float): Attention dropout rate. Default: 0
        drop_path_rate (float): Stochastic depth rate. Default: 0.1
        norm_layer (nn.Module): Normalization layer. Default: nn.LayerNorm.
        ape (bool): If True, add absolute position embedding to the patch embedding. Default: False
        patch_norm (bool): If True, add normalization after patch embedding. Default: True
        use_checkpoint (bool): Whether to use checkpointing to save memory. Default: False
        pretrained_window_sizes (tuple(int)): Pretrained window sizes of each layer.
        diff_attn_layers (list(int)): Layers to use dynamic attention mechanism.
    """

    def __init__(self, img_size=384, patch_size=4, in_chans=3, num_classes=1,
                 embed_dim=96, depths=[2, 2, 6, 2], num_heads=[3, 6, 12, 24],
                 window_size=12, mlp_ratio=4., qkv_bias=True,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0.1,
                 norm_layer=nn.LayerNorm, ape=False, patch_norm=True,
                 use_checkpoint=False, pretrained_window_sizes=[0, 0, 0, 0],
                 diff_attn_layers = [1], **kwargs):
        super().__init__()

        self.num_classes = num_classes
        self.num_layers = len(depths)
        self.embed_dim = embed_dim
        self.ape = ape
        self.patch_norm = patch_norm
        self.num_features = int(embed_dim * 2 ** (self.num_layers - 1))
        self.mlp_ratio = mlp_ratio

        # split image into non-overlapping patches
        self.patch_embed_1 = PatchEmbed(
            img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim,
            norm_layer=norm_layer if self.patch_norm else None)
        self.patch_embed_2 = PatchEmbed(
            img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim,
            norm_layer=norm_layer if self.patch_norm else None)
        num_patches = self.patch_embed_1.num_patches
        patches_resolution = self.patch_embed_1.patches_resolution
        self.patches_resolution = patches_resolution

        # absolute position embedding
        if self.ape:
            self.absolute_pos_embed_1 = nn.Parameter(
                torch.zeros(1, num_patches, embed_dim))
            self.absolute_pos_embed_2 = nn.Parameter(
                torch.zeros(1, num_patches, embed_dim))
            trunc_normal_(self.absolute_pos_embed_1, std=.02)
            trunc_normal_(self.absolute_pos_embed_2, std=.02)

        self.pos_drop_1 = nn.Dropout(p=drop_rate)
        self.pos_drop_2 = nn.Dropout(p=drop_rate)

        # stochastic depth
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate,
                                                sum(depths))]  # stochastic depth decay rule

        # build multiview layers
        self.layers = nn.ModuleList()
        for i_layer in range(self.num_layers // 2):
            layer = BasicLayer_multiview(dim=int(embed_dim * 2 ** i_layer),
                               input_resolution=(patches_resolution[0] // (2 ** i_layer),
                                                 patches_resolution[1] // (2 ** i_layer)),
                               depth=depths[i_layer],
                               num_heads=num_heads[i_layer],
                               window_size=window_size,
                               mlp_ratio=self.mlp_ratio,
                               qkv_bias=qkv_bias,
                               drop=drop_rate, attn_drop=attn_drop_rate,
                               drop_path=dpr[sum(depths[:i_layer]):sum(
                                   depths[:i_layer + 1])],
                               norm_layer=norm_layer,
                               downsample=PatchMerging if (
                                   i_layer < self.num_layers - 1) else None,
                               use_checkpoint=use_checkpoint,
                               pretrained_window_size=pretrained_window_sizes[i_layer],
                               diff_attn=True if i_layer in diff_attn_layers else False)
            self.layers.append(layer)

        fused_in_dim = patches_resolution[0] // (2 **
                                                 i_layer) * window_size * 2
        fused_out_dim = fused_in_dim // 2
        self.fc_layer_fused = nn.Linear(fused_in_dim, fused_out_dim)

        # build fused layers
        self.layers_fused = nn.ModuleList()
        for i_layer in range(2, self.num_layers):
            layer = BasicLayer_singleview(dim=int(embed_dim * 2 ** i_layer),
                                        input_resolution=(patches_resolution[0] // (2 ** i_layer),
                                                          patches_resolution[1] // (2 ** i_layer)),
                                        depth=depths[i_layer],
                                        num_heads=num_heads[i_layer],
                                        window_size=window_size,
                                        mlp_ratio=self.mlp_ratio,
                                        qkv_bias=qkv_bias,
                                        drop=drop_rate, attn_drop=attn_drop_rate,
                                        drop_path=dpr[sum(depths[:i_layer]):sum(
                                            depths[:i_layer + 1])],
                                        norm_layer=norm_layer,
                                        downsample=PatchMerging if (
                                            i_layer < self.num_layers - 1) else None,
                                        use_checkpoint=use_checkpoint,
                                        pretrained_window_size=pretrained_window_sizes[i_layer])
            self.layers_fused.append(layer)

        self.norm = norm_layer(self.num_features)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.head = nn.Linear(
            self.num_features, num_classes) if num_classes > 0 else nn.Identity()
        self.act = nn.Sigmoid()

        self.apply(self._init_weights)
        for bly in self.layers:
            bly._init_respostnorm()

        for bly in self.layers_fused:
            bly._init_respostnorm()

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'absolute_pos_embed'}

    @torch.jit.ignore
    def no_weight_decay_keywords(self):
        return {"cpb_mlp", "logit_scale", 'relative_position_bias_table'}

    def forward_features(self, x1, x2):
        x1 = self.patch_embed_1(x1)
        x2 = self.patch_embed_2(x2)
        if self.ape:
            x1 = x1 + self.absolute_pos_embed_1
            x2 = x2 + self.absolute_pos_embed_2

        x1 = self.pos_drop_1(x1)
        x2 = self.pos_drop_2(x2)

        for layer in self.layers:
            x1, x2 = layer(x1, x2)

        x = torch.cat((x1, x2), dim=1).permute(0, 2, 1)
        # print("fc_fuse: ", x1.shape, x2.shape, x.shape, int(self.embed_dim * 2 ** (self.num_layers - 3)), self.patches_resolution[0] // (2 ** 1), self.patches_resolution[1] // (2 ** 1))
        x = self.fc_layer_fused(x).permute(0, 2, 1)

        for layer in self.layers_fused:
            x = layer(x)

        x = self.norm(x)
        x = self.avgpool(x.transpose(1, 2))
        x = torch.flatten(x, 1)
        return x

    def forward(self, x1, x2):
        x = self.forward_features(x1, x2)
        x = self.head(x)
        return self.act(x)

In [ ]:
import torch
print(torch.cuda.is_available())  # Should print True if CUDA is available

True


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Custom Dataset Class
class BreastCancerDataset(Dataset):
    def __init__(self, benign_dir, malignant_dir, transform=None):
        self.benign_images = [os.path.join(benign_dir, img) for img in os.listdir(benign_dir)]
        self.malignant_images = [os.path.join(malignant_dir, img) for img in os.listdir(malignant_dir)]
        self.images = self.benign_images + self.malignant_images
        self.labels = [0] * len(self.benign_images) + [1] * len(self.malignant_images)  # 0 for benign, 1 for malignant
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((384, 384)),  # Resize to the input size of the model
    transforms.ToTensor(),
])

# Load dataset
benign_dir = '/content/Dataset/Benign'  # Update with your path
malignant_dir = '/content/Dataset/Malignant'  # Update with your path
dataset = BreastCancerDataset(benign_dir, malignant_dir, transform)

# Split dataset into training and testing
train_images, test_images, train_labels, test_labels = train_test_split(
    dataset.images, dataset.labels, test_size=0.2, random_state=42
)

# Create DataLoader
train_dataset = BreastCancerDataset(benign_dir, malignant_dir, transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = BreastCancerDataset(benign_dir, malignant_dir, transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Initialize the model
model = MVSwinTransformer(num_classes=1)  # Assuming binary classification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device).float()
        optimizer.zero_grad()
        outputs = model(images, images)  # Assuming you are using the same images for both views
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        predicted = (outputs.squeeze() > 0.5).float()  # Thresholding at 0.5
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate average loss and accuracy
    avg_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')

# Evaluation loop
model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device).float()
        outputs = model(images, images)
        predicted = (outputs.squeeze() > 0.5).float()  # Thresholding at 0.5
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_preds.extend(predicted.cpu().numpy())  # Collect predictions
        all_labels.extend(labels.cpu().numpy())  # Collect true labels

# Calculate Accuracy
accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

# Calculate additional evaluation metrics
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
conf_matrix = confusion_matrix(all_labels, all_preds)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Confusion Matrix:\n{conf_matrix}')


/usr/local/lib/python3.11/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 259/259 [06:02<00:00,  1.40s/it]


Epoch [1/10], Loss: 0.6899, Accuracy: 52.48%


100%|██████████| 259/259 [06:07<00:00,  1.42s/it]


Epoch [2/10], Loss: 0.6496, Accuracy: 59.07%


100%|██████████| 259/259 [06:07<00:00,  1.42s/it]


...

Epoch [8/10], Loss: 0.1757, Accuracy: 93.92%


100%|██████████| 259/259 [06:07<00:00,  1.42s/it]


Epoch [9/10], Loss: 0.0841, Accuracy: 97.63%


100%|██████████| 259/259 [06:07<00:00,  1.42s/it]


Epoch [10/10], Loss: 0.0451, Accuracy: 98.86%
Test Accuracy: 99.93%
Precision: 0.9990
Recall: 0.9995
F1 Score: 0.9993
Confusion Matrix:
[[2057    2]
 [   1 2071]]
